In [1]:
import sys
import logging
import os
import json
import pandas as pd

date_strftime_format = "%Y-%m-%y %H:%M:%S"
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(asctime)s %(message)s", datefmt=date_strftime_format)

# LOAD DATA

In [2]:
DATA_FOLDER = os.getcwd() + '/review_scorer/data/'
SENTI_PATH = DATA_FOLDER + 'SentiWord_info.json'
DATA_PATH = DATA_FOLDER + 'data_origin.csv'
data: pd.DataFrame

with open(SENTI_PATH, mode='rt', encoding='UTF8') as f:
    senti = pd.DataFrame.from_dict(json.load(f))

data = pd.read_csv(DATA_PATH, encoding='UTF8')
data = data.dropna(axis=0)
data = data.sample(frac=1).reset_index(drop=True)

# TOKENIZER

In [3]:
from tqdm import tqdm
from twkorean import TwitterKoreanProcessor

processor = TwitterKoreanProcessor()
tokenize = processor.tokenize_to_strings
tokens = [tokenize(_) for _ in tqdm(data.review)]

100%|████████████████████████████████████| 29557/29557 [03:25<00:00, 143.76it/s]


# SET REVIEW SCORER

In [4]:
from review_scorer import Doc2Category

# Review scorer needs tokens of datas to train when initializing it.
# 리뷰 채점기 클래스를 생성할 때, 토크나이즈 된 데이터를 인자로 주어야 합니다.
model = Doc2Category(sentences=tokens, senti_dict_path=SENTI_PATH)

2022-08-22 13:43:36 collecting all words and their counts
2022-08-22 13:43:36 PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


/home/piai/anaconda3/envs/pytorch/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2022-08-22 13:43:42 PROGRESS: at sentence #10000, processed 1119791 words, keeping 30365 word types
2022-08-22 13:43:49 PROGRESS: at sentence #20000, processed 2237102 words, keeping 42611 word types
2022-08-22 13:43:55 collected 51667 word types from a corpus of 3315605 raw words and 29557 sentences
2022-08-22 13:43:55 Creating a fresh vocabulary
2022-08-22 13:43:55 Doc2Category lifecycle event {'msg': 'effective_min_count=5 retains 15230 unique words (29.48% of original 51667, drops 36437)', 'datetime': '2022-08-17T13:43:55.372670', 'gensim': '4.2.0', 'python': '3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) \n[GCC 9.4.0]', 'platform': 'Linux-5.4.0-65-generic-x86_64-with-debian-buster-sid', 'event': 'prepare_vocab'}
2022-08-22 13:43:55 Doc2Category lifecycle event {'msg': 'effective_min_count=5 leaves 3260723 word corpus (98.34% of original 3315605, drops 54882)', 'datetime': '2022-08-17T13:43:55.373568', 'gensim': '4.2.0', 'python': '3.7.12 | packaged by conda-f

## TAGGING RIVIEW SCORER's SENTIMENTAL DICTIONARY

In [10]:
# Tagging review scorer's sentimental dictionary by category.
# 카테고리에 따라 리뷰 채점기의 감성사전을 태깅합니다.
model.tag(categories={'taste': ['맛', '맛있다', '맛없다'],
                      'price': ['가격', '싸다', '비싸다', '저렴'],
                      'service': ['서비스', '친절', '싸가지'],
                      'atmosphere': ['인테리어', '분위기']},
          width=6, depth=6)

polarity: 100%|██████████████████████████| 14854/14854 [00:32<00:00, 456.91it/s]


# SCORING WITH REVIEW SCORER

In [17]:
start = 2000
for i in range(start, start + 10):
    print('index: ', i)
    print(model.score_review(tokenize(data.review.iloc[i])))
    print(data.review.iloc[i])
    print()

index:  2000
{'taste': 8.057142857142857, 'price': 1.0, 'service': -2.0, 'atmosphere': 0.0}
한줄평: 가성비 런치 스시도 좋지만, 요리 중심 디너 코스도 최고!런치에 맛있게 스시를 먹었던 와려에 디너에 방문했다. 디너도 스시 쥐어주시는 줄 알았는데, 디너는 요리 중심이었다. 원래도 디너부터 영업했다가 런치 시간이 비니까 스시를 시작하셨던 거라고 한다.- 호박스프하몽 슬라이스가 올라간 호박스프. 마치 스페인에서 먹던 가스파초의 어레인지 버전 같은 맛이다.- 광어 딱새우 가리비 샐러드내가 좋아하는 해산물들이 가득 들어간 샐러드. 광어도 탱글하고 달큰한 딱새우와 부드러운 가리비 최고다!- 우니크림면충격적으로 맛있었던 디쉬. 일본 3대 면이라 불리는 이나니와면을 사용해서 크림소스면. 위에 우니와 캐비어가 올라감. 우니를 잘 으깨 비벼먹어야한다. 우니의 녹진한 맛과 크리미한 이나니와면이 정말 잘 어울려❤- 모듬사시미왼쪽부터 참돔 참치뱃살 잿방어 능섬어 시마아지 밴자리돔. 고급 어종이 가득한 사시미! 네타 상태도 좋고 전부 맛잇다- 맑은국노래미가 들어간 맑은 국- 청보리 리조또전복 내장을 사용한 리조또다. 와려 디너 코스의 시그니처 메뉴 중 하나. 전복 너무 쫜득야들하고, 내장 리조또도 식감이나 맛이나 진짜 맛있다!! 감탄사가 절로남(엄지척)❤- 아지나메로나메로는 일본 치바지역 향토 조리방식으로, 아지나메로는 전갱이를 구워서 잘게 다져서 양념에 버무린 것이다. 와려에서는 된장과 시소랑 버무렸는데, 요걸 김에 싸먹으니 마치 군함마키를 손으로 싸먹는 기분!- 은대구 간장구이중간에 구이 등장. 은대구의 야들야들하고 탱탱한 살점은 말하지 않아도 맛있음. 껍질까지 맛있...- 소고기 시소말이 튀김위에는 우엉튀김이 올라가고, 아래 미소소스가 뿌려졌다. 시소가 향긋하게 향을 내면서 소고기 육즙을 잡아준다. 우엉이 바삭한 식감을 배가시킴.- 장어솥밥비주얼 끝판왕❤ 넘나 먹음직스러운 장어솥밥! 처음에 잘 섞어서 밥을